In [1]:
import os
import math
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization, LeakyReLU, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
tf.random.set_seed(42)

In [2]:

def smooth_l1_loss(y_true, y_pred):
    huber = tf.keras.losses.Huber(delta=1.0)  # Smooth L1 Loss
    return huber(y_true, y_pred)

In [16]:
import cv2
import numpy as np
import tensorflow as tf
import time
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def smooth_l1_loss(y_true, y_pred, delta=1.0):
    abs_diff = tf.abs(y_true - y_pred)
    loss = tf.where(abs_diff < delta, 0.5 * abs_diff ** 2 / delta, abs_diff - 0.5 * delta)
    return tf.reduce_mean(loss)

class WeaponDetectionSystem:
    def __init__(self, model_path: str, confidence_threshold: float = 0.5):
        self.model = self._load_model(model_path)
        self.confidence_threshold = confidence_threshold
        self.frame_count = 0
        self.fps = 0
        self.start_time = time.time()
        self.class_labels = ['Knife', 'Gun', 'LongGun']

    def _load_model(self, model_path: str) -> tf.keras.Model:
        try:
            model = tf.keras.models.load_model(model_path, custom_objects={'smooth_l1_loss': smooth_l1_loss})
            logger.info(f"Model loaded successfully. Input shape: {model.input_shape}")
            return model
        except Exception as e:
            logger.error(f"Failed to load model: {str(e)}")
            raise

    def preprocess_frame(self, frame: np.ndarray) -> np.ndarray:
        """Normalize and expand dims (no resizing needed)."""
        try:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_array = frame_rgb.astype(np.float32) / 255.0
            frame_array = np.expand_dims(frame_array, axis=0)
            return frame_array
        except Exception as e:
            logger.error(f"Frame preprocessing failed: {str(e)}")
            raise

    def draw_bounding_box(self, frame: np.ndarray, box: np.ndarray, label: str) -> np.ndarray:
        """Draw bounding box on 416x416 frame."""
        img_height, img_width = frame.shape[:2]
        xmin, ymin, xmax, ymax = box

        # Convert normalized to absolute pixel coordinates
        xmin = int(xmin * img_width)
        ymin = int(ymin * img_height)
        xmax = int(xmax * img_width)
        ymax = int(ymax * img_height)

        # Clip to image boundaries
        xmin = max(0, xmin)
        ymin = max(0, ymin)
        xmax = min(img_width, xmax)
        ymax = min(img_height, ymax)

        # Draw box
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)

        # Draw label
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        font_thickness = 2
        (text_width, text_height), _ = cv2.getTextSize(label, font, font_scale, font_thickness)

        cv2.rectangle(frame, (xmin, ymin - text_height - 10), (xmin + text_width, ymin), (0, 0, 0), -1)
        cv2.putText(frame, label, (xmin, ymin - 5), font, font_scale, (0, 255, 0), font_thickness)

        return frame

    def run_detection(self):
        try:
            cap = cv2.VideoCapture(0)

            if not cap.isOpened():
                raise ValueError("Failed to open video capture device")

            while True:
                ret, frame = cap.read()
                if not ret:
                    logger.error("Failed to read frame from camera")
                    break

                # Resize camera frame to 416x416 immediately
                frame = cv2.resize(frame, (416, 416))

                # FPS calculation
                self.frame_count += 1
                if (time.time() - self.start_time) > 1:
                    self.fps = self.frame_count / (time.time() - self.start_time)
                    self.frame_count = 0
                    self.start_time = time.time()

                # Preprocess (only normalize and expand dims)
                frame_array = self.preprocess_frame(frame)

                # Prediction
                predictions = self.model.predict(frame_array, verbose=0)

                if self.frame_count % 30 == 0:
                    logger.info(f"Predictions: {predictions}")

                if len(predictions) == 2:
                    class_probs, boxes = predictions
                    class_probs = class_probs[0]  # shape (3,)
                    boxes = boxes[0]              # shape (4,)

                    max_class_idx = np.argmax(class_probs)
                    max_score = class_probs[max_class_idx]

                    if max_score > self.confidence_threshold:
                        label = f"{self.class_labels[max_class_idx]}: {max_score:.2f}"
                        frame = self.draw_bounding_box(frame, boxes, label)

                # Show 416x416 frame
                cv2.imshow('Weapon Detection System', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        except Exception as e:
            logger.error(f"Error during detection: {str(e)}")
        finally:
            cap.release()
            cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = WeaponDetectionSystem(
        model_path='model/custom_25/model-45-3.14.h5',
        confidence_threshold=0.9
    )
    detector.run_detection()


INFO:__main__:Model loaded successfully. Input shape: (None, 416, 416, 3)
INFO:__main__:Predictions: [array([[0.68803304, 0.4918543 , 0.01846379]], dtype=float32), array([[0.2989387 , 0.25188735, 0.6091539 , 0.6368888 ]], dtype=float32)]
INFO:__main__:Predictions: [array([[0.95889276, 0.03005708, 0.02265255]], dtype=float32), array([[0.31712762, 0.23718642, 0.54693854, 0.6450284 ]], dtype=float32)]
INFO:__main__:Predictions: [array([[0.94664186, 0.0530348 , 0.01857813]], dtype=float32), array([[0.3180228 , 0.24119288, 0.56618375, 0.67794317]], dtype=float32)]
INFO:__main__:Predictions: [array([[0.94290155, 0.03748811, 0.0306229 ]], dtype=float32), array([[0.3132164 , 0.2272695 , 0.56419116, 0.6668351 ]], dtype=float32)]
INFO:__main__:Predictions: [array([[0.9367838 , 0.01957442, 0.05740381]], dtype=float32), array([[0.2705869 , 0.2292537 , 0.57300764, 0.6699204 ]], dtype=float32)]
INFO:__main__:Predictions: [array([[0.9601429 , 0.02208247, 0.02867456]], dtype=float32), array([[0.283687